In [ ]:
### Fixing the coding of reference.
import os

code_path = "/Users/alejandrodelaconcha/Desktop/Doctorat/projects/Paper Change on Mean/research code"  ### Change directory
os.chdir(code_path)

In [ ]:
### Import code generating the scenarios.
from simulation_environment.scenarios_generator import *
from models.variable_selection_detector import *
from valuation_metrics.valuation_metrics import *

# Example using the first scenario described in the paper.

## Generate dataset

In [ ]:
#### Parameters

# Input.
# 1) The number of change-points is generated via a Poisson distribution with mean "mean_change-points".
# 2) The distance between change-points is at least 30 + an exponential distribution with mean "mean_exponential".
# 3) An Erdos Graph with "n_nodes" and probability "p_erdos" is generated.
# 4) The spectral profile of the filter is np.sqrt(15)/(np.log(x+10)+1).
# 5) The lowest "fixed_frequencies" are generated at random previous to the first change-point.
# 6) A given number "random_frequencies" are selected at random and they are modified after each change-point.

n_nodes = 500
fixed_frequencies = 100
random_frequencies = 20
mean_change_points = 5
mean_exponential = 20
p_erdos = 0.3

In [ ]:
### Generating the First Scenario.
signal, GSO, change_points, mu, PSD, G = Scenario_I(
    n_nodes,
    mean_change_points,
    fixed_frequencies,
    random_frequencies,
    mean_exponential,
    p_erdos,
)

Other datasets can be plugged. They only require to be numpy arrays of dimention Txp, where T is the number of Graph Signals and p is the number of nodes. 

## Kernel method logic.

The idea of the algorithm is to obtain the number the change point detection problem by solving the penalized problem:

\begin{align}
\hat{d}=\operatorname{argmin}_{ d \in \{1,...,D_{max} \} } C(y,d) + pen(d), 
\end{align}

where $C(y,d)$ is the value of the cost function evaluated at the best partition of lenght $d$ and $pen(d)$ is a partition for the number of change-points:

\begin{align}
pen(d)= \frac{d}{N}(c_1 + c_2 {{N-1}\choose{d-1}} )
\end{align}

The paper A Kernel Multiple Change-point Algorithm via Model Selection Arlot aims to recover the constants $c_1$ and $c_2$. 


This is done using a linear regression of $C(y,d)$ against the terms $\frac{d}{N}$ and $\frac{d}{N}{{N-1}\choose{d-1}}$ and multipling the results by -2. 

Here I put explicetly the code so can see directly how it work in practice and think about how can we added to the library. 

In [ ]:
import ruptures as rpt
import numpy as np
from sklearn.linear_model import LinearRegression
import scipy


class Kernel_model_selection:
    def __init__(self, D_max, cost_fuction, coefs=None):

        ##### Function initializing the class

        ### Input

        ## D_max = maximum number of change-points to chacke
        ## cost_function = cost function object from the ruptures library
        ## coefs: list with the coeficients for the cost_function ralated with the number of change-points/

        self.D_max = D_max
        self.cost_function = cost_fuction
        self.coefs = coefs

    def fit(self, data):

        #### Function which finds the diferent partitions from size 0 up to size D_max

        ### Input
        ## data= matrix of size Txp where T is the time horizon, p the number of nodes.

        self.change_points = []
        self.data = data
        self.T = data.shape[0]

        self.change_points.append([self.T])

        ## Solving the change-point detection problem usig Dynamic Programming.

        detector = rpt.Dynp(custom_cost=self.cost_function, min_size=2, jump=2).fit(
            data
        )

        for d in range(1, self.D_max):
            self.change_points.append(detector.predict(n_bkps=d))

    def predict(self):
        ### Function selecting the optimal number of change-points via a model selection approach.

        ### Output
        ## optimal partition.

        self.get_partitions_cost()
        if self.coefs is None:
            self.get_constants()
        log_comb = np.array(
            [self.log_comb(self.T - 1, d) for d in np.arange(0, self.D_max)]
        )
        index = np.argmin(
            self.partition_cost
            + (1.0 / self.T)
            * (self.coefs[0] * log_comb + self.coefs[1] * np.arange(1, self.D_max + 1))
        )

        self.predicted_change_points = self.change_points[index]

        return self.predicted_change_points

    def get_partitions_cost(self):
        #### Function estimating the total cost for a given partition.

        self.partition_cost = np.zeros(self.D_max)

        self.partition_cost[0] = self.cost_function.error(0, self.change_points[0][0])
        for d in range(1, len(self.change_points)):

            self.partition_cost[d] = self.cost_function.error(
                0, self.change_points[d][0]
            )

            for i in range(1, len(self.change_points[d])):

                self.partition_cost[d] += self.cost_function.error(
                    self.change_points[d][i - 1], self.change_points[d][i]
                )

    def get_constants(self, lower_bound=None):

        #### Function computing the constants which are required for the computation of the
        ## optimal number of change-points (This does the linear regression )

        ### Input
        ## lower_bound: left side of the interval where the regression is done.

        if lower_bound is None:
            lower_bound = np.floor(0.6 * self.D_max)

        rank_D = np.arange(lower_bound, self.D_max + 1)
        y = self.partition_cost[[int(x) - 1 for x in rank_D]]
        log_comb = np.array([self.log_comb(self.T - 1, d) for d in (rank_D - 1)])
        x = np.array((log_comb / self.T, rank_D / self.T)).transpose()
        ### The coefficients are obtained via robust linear regression.
        self.coefs = LinearRegression().fit(x, y).coef_
        self.coefs = -2.0 * self.coefs

    def log_comb(self, T, d):
        ### This function estimates the log combinations of T in n to avoid numerical errors.
        return np.sum(np.log(np.arange(d + 1, T + 1))) - np.sum(
            np.log(np.arange(1, T - d + 1))
        )

## Change-point detection using Kernel Methods. 

1) Fix the right parameters. 

In [ ]:
T = signal.shape[0]
D_max = int(T / np.log(T))

In [ ]:
linear_kernel_cost = rpt.costs.CostMl(metric=np.eye(n_nodes)).fit(
    signal
)  ## Linear Kernel
laplacian_kernel_cost = rpt.costs.CostMl(metric=GSO).fit(signal)  ## Laplacian Kernel
gaussian_kernel_cost = rpt.costs.CostRbf().fit(signal)  ## Gaussian Kernel

2) Train each of the models and predict the change-points. 

In [ ]:
### Linear Kernel
linear_kernel_detector = Kernel_model_selection(D_max, linear_kernel_cost)
linear_kernel_detector.fit(signal)
change_points_linear = linear_kernel_detector.predict()

In [ ]:
## Laplacian Kernel
laplacian_kernel_detector = Kernel_model_selection(D_max, laplacian_kernel_cost)
laplacian_kernel_detector.fit(signal)
change_points_laplacian = laplacian_kernel_detector.predict()

In [ ]:
## Gaussian Kernel
Gaussian_kernel_detector = Kernel_model_selection(D_max, gaussian_kernel_cost)
Gaussian_kernel_detector.fit(signal)
change_points_gaussian = Gaussian_kernel_detector.predict()

3) Evaluate the performance of each model. 

In [ ]:
(
    haussdorf_distance_linear,
    randindex_linear,
    precision_linear,
    recall_linear,
    F1_linear,
) = get_valuation_metrics(change_points, change_points_linear)
print("     Linear Kernel     ")
print("real_change_points: " + str(change_points))
print("estimated_change_points: " + str(change_points_linear))
print("Haussdor distance: " + str(haussdorf_distance_linear))
print("Randindex: " + str(randindex_linear))
print("Precision: " + str(precision_linear))
print("Recall: " + str(recall_linear))
print("F1: " + str(F1_linear))

In [ ]:
(
    haussdorf_distance_laplacian,
    randindex_laplacian,
    precision_laplacian,
    recall_laplacian,
    F1_laplacian,
) = get_valuation_metrics(change_points, change_points_laplacian)
print("     Laplacian Kernel     ")
print("real_change_points: " + str(change_points))
print("estimated_change_points: " + str(change_points_laplacian))
print("Haussdor distance: " + str(haussdorf_distance_laplacian))
print("Randindex: " + str(randindex_laplacian))
print("Precision: " + str(precision_laplacian))
print("Recall: " + str(recall_laplacian))
print("F1: " + str(F1_laplacian))

In [ ]:
(
    haussdorf_distance_gaussian,
    randindex_gaussian,
    precision_gaussian,
    recall_gaussian,
    F1_gaussian,
) = get_valuation_metrics(change_points, change_points_gaussian)
print("     Gaussian Kernel     ")
print("real_change_points: " + str(change_points))
print("estimated_change_points: " + str(change_points_gaussian))
print("Haussdor distance: " + str(haussdorf_distance_gaussian))
print("Randindex: " + str(randindex_gaussian))
print("Precision: " + str(precision_gaussian))
print("Recall: " + str(recall_gaussian))
print("F1: " + str(F1_gaussian))